In [ ]:
import torch
import os

# https://www.youtube.com/watch?v=E5kzAbD8D0w, DPO
# https://www.youtube.com/watch?v=wybjsKtA9hI, SQL Backend
# https://www.youtube.com/watch?v=CZw57SIwgiE, Threads and Connections
# https://www.youtube.com/watch?v=p6wwj0ozifw, Hashing
# https://www.youtube.com/watch?v=4NB0NDtOwIQ, sys_design
# https://www.youtube.com/watch?v=kxT8-C1vmd4, zig
# https://www.youtube.com/watch?v=5C_HPTJg5ek, rust
# https://www.youtube.com/watch?v=HyWYpM_S-2c, React
# https://www.youtube.com/watch?v=MnpuK0MK4yo, react_svelte
# https://www.youtube.com/watch?v=mE7IDf2SmJg, stanford_lecture_rag
# https://www.youtube.com/watch?v=utYC2tKDdoQ, umich_os_locks
# https://www.youtube.com/watch?v=YAgjfMR9R_M, umich_dl_attention
# https://www.youtube.com/watch?v=dsHyUgGMht0, k_fang_github_break 


videos = [
    ["https://www.youtube.com/watch?v=UiX8K-xBUpE", "UiX8K-xBUpE", "mixtral8x7b"],
    ["https://www.youtube.com/watch?v=Xdhe-meS30A", "Xdhe-meS30A", "catan"],
    ["https://www.youtube.com/watch?v=00AQjBDM46Q", "00AQjBDM46Q", "mlops_llm_eval"],
    ["https://www.youtube.com/watch?v=5ChkQKUzDCs", "5ChkQKUzDCs", "typescript_fireship"],
    ["https://www.youtube.com/watch?v=J0NuOlA2xDc", "J0NuOlA2xDc", "localized_deployment"],
    ["https://www.youtube.com/watch?v=E5kzAbD8D0w", "E5kzAbD8D0w", "dpo"],
    ["https://www.youtube.com/watch?v=wybjsKtA9hI", "wybjsKtA9hI", "sql_backend"],
    ["https://www.youtube.com/watch?v=CZw57SIwgiE", "CZw57SIwgiE", "threads_connections"],
    ["https://www.youtube.com/watch?v=p6wwj0ozifw", "p6wwj0ozifw", "hashing"],
    # ["https://www.youtube.com/watch?v=4NB0NDtOwIQ", "4NB0NDtOwIQ", "sys_design"], // Note this failed due to not having a timestamp every line
    ["https://www.youtube.com/watch?v=kxT8-C1vmd4", "kxT8-C1vmd4", "zig"],
    ["https://www.youtube.com/watch?v=5C_HPTJg5ek", "5C_HPTJg5ek", "rust"],
    ["https://www.youtube.com/watch?v=HyWYpM_S-2c", "HyWYpM_S-2c", "react"],
    ["https://www.youtube.com/watch?v=MnpuK0MK4yo", "MnpuK0MK4yo", "react_svelte"],
]

name_to_url = {
    "mixtral8x7b": "https://www.youtube.com/watch?v=UiX8K-xBUpE",
    "catan": "https://www.youtube.com/watch?v=Xdhe-meS30A",
    "mlops_llm_eval": "https://www.youtube.com/watch?v=00AQjBDM46Q",
    "typescript_fireship": "https://www.youtube.com/watch?v=5ChkQKUzDCs",
    "localized_deployment": "https://www.youtube.com/watch?v=J0NuOlA2xDc",
    "dpo": "https://www.youtube.com/watch?v=E5kzAbD8D0w",
    "sql_backend": "https://www.youtube.com/watch?v=wybjsKtA9hI",
    "threads_connections": "https://www.youtube.com/watch?v=CZw57SIwgiE",
    "hashing": "https://www.youtube.com/watch?v=p6wwj0ozifw",
    "zig": "https://www.youtube.com/watch?v=kxT8-C1vmd4",
    "rust": "https://www.youtube.com/watch?v=5C_HPTJg5ek",
    "react": "https://www.youtube.com/watch?v=HyWYpM_S-2c",
    "react_svelte": "https://www.youtube.com/watch?v=MnpuK0MK4yo",
}


In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key="git ")
index = pc.Index("inftest")

In [ ]:
vectors = []
for embedding_file in os.listdir('data/embeddings'):
    if not embedding_file.endswith('.pt'):
        continue
    name = embedding_file[:-3]
    metadata = {"name": name, "url": name_to_url[name]}
    tensor = torch.load(f'data/embeddings/{embedding_file}')
    # max_pooled = torch.max(tensor, 1).squeeze(0)
    avg_pooled = torch.mean(tensor, 1).squeeze(0)
    vectors.append({"values": avg_pooled.numpy().tolist(), "id": name, "metadata": metadata})


In [ ]:
index.upsert(vectors=vectors)

In [ ]:

for embedding_file in os.listdir('data/embeddings'):
    if not embedding_file.endswith('.pt'):
        continue
    name = embedding_file[:-3]
    # Tensor is shape[1, 968, 4096]
    tensor = torch.load(f'data/embeddings/{embedding_file}')
    max_pooled = torch.max(tensor, 1)
    average_pool = torch.mean(tensor, 1)
    query = average_pool.squeeze(0).numpy().tolist()
    response = index.query(vector=query, top_k=5, include_values=True, include_metadata=True)
    print(f'Querying {name}:')
    for i, obj in enumerate(response["matches"]):
        print(f'    Rank: {i+1}, Metadata: {obj["metadata"]}, Distance:, Score: {obj["score"]}')
